In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.datasets as datasets
#from vae import VAE
from vaecnn import VAECNN
#from util import train,test

In [2]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    #BCE = F.binary_cross_entropy(recon_x, x, reduction='sum') #.view(-1, 784)
    BCE = F.mse_loss(recon_x, x, size_average=False) / 150
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = (-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()))/ (150)
    
    print('BCE',BCE)
    print('KLD',KLD)
    
    return  BCE + 0.01 * (KLD )


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        #print(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar,_ = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar, _ = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
#             if i == 0:
#                 n = min(data.size(0), 8)
#                 comparison = torch.cat([data[:n],
#                                       recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
#                 save_image(comparison.cpu(),
#                          'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
args = Namespace(
    batch_size = 150,
    epochs = 10,
    cuda = True,
    seed = 2019,
    log_interval = 1)

In [5]:
torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

trainDataDir = '/scratch/um367/DL/data/sample_4/train'
valDataDir = '/scratch/um367/DL/data/sample_4/val'
# trainDataDir = '/scratch/um367/DL/data/sampledata/supervised/train'
# valDataDir = '/scratch/um367/DL/data/sampledata/supervised/val'


train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(trainDataDir, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valDataDir, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [7]:
print(len(train_loader.dataset))

128000


In [8]:
model = VAECNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
## load parameters
# to load
checkpoint = torch.load('vaetest3.pth.tar')
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    #test(epoch)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'vaetest3.pth.tar')
#     with torch.no_grad():
#         sample = torch.randn(64, 20).to(device)
#         sample = model.decode(sample).cpu()
#         save_image(sample.view(64, 1, 28, 28),
#         'results/sample_' + str(epoch) + '.png')

/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BCE tensor(552.2264, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.3819, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [0/128000 (0%)]	Loss: 3.699601
BCE tensor(2002.8623, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(299.8569, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [150/128000 (0%)]	Loss: 13.372406
BCE tensor(723.6709, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(264.9092, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [300/128000 (0%)]	Loss: 4.842133
BCE tensor(708.6439, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(243.3640, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [450/128000 (0%)]	Loss: 4.740517
BCE tensor(888.0856, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(236.4135, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [600/128000 (0%)]	Loss: 5.936332
BCE tensor(850.8558, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(229.0626, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [

Train Epoch: 1 [7050/128000 (6%)]	Loss: 3.885825
BCE tensor(560.4083, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(310.5019, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [7200/128000 (6%)]	Loss: 3.756756
BCE tensor(574.0785, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(304.6717, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [7350/128000 (6%)]	Loss: 3.847501
BCE tensor(548.4348, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(313.8286, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [7500/128000 (6%)]	Loss: 3.677154
BCE tensor(533.0632, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(308.9372, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [7650/128000 (6%)]	Loss: 3.574350
BCE tensor(551.2176, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(304.2804, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [7800/128000 (6%)]	Loss: 3.695069
BCE tensor(570.6559, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(309.0182, dev

BCE tensor(563.3452, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.8204, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [14250/128000 (11%)]	Loss: 3.773756
BCE tensor(505.5283, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(273.6667, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [14400/128000 (11%)]	Loss: 3.388433
BCE tensor(516.1605, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(280.4745, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [14550/128000 (11%)]	Loss: 3.459768
BCE tensor(556.7616, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(276.8505, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [14700/128000 (11%)]	Loss: 3.730201
BCE tensor(496.6679, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(277.0527, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [14850/128000 (12%)]	Loss: 3.329589
BCE tensor(531.8563, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(274.2789, device='cuda:0', grad_fn=<DivBackward0>)
T

BCE tensor(542.9590, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(273.4746, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [21300/128000 (17%)]	Loss: 3.637958
BCE tensor(506.8703, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(276.5028, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [21450/128000 (17%)]	Loss: 3.397569
BCE tensor(532.5193, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(274.7695, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [21600/128000 (17%)]	Loss: 3.568446
BCE tensor(502.9397, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(275.3576, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [21750/128000 (17%)]	Loss: 3.371288
BCE tensor(523.3586, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(273.9501, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [21900/128000 (17%)]	Loss: 3.507321
BCE tensor(527.1189, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.2020, device='cuda:0', grad_fn=<DivBackward0>)
T

...

Train Epoch: 1 [119850/128000 (94%)]	Loss: 3.476431
BCE tensor(500.5799, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(268.6512, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [120000/128000 (94%)]	Loss: 3.355109
BCE tensor(507.1019, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(268.4020, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [120150/128000 (94%)]	Loss: 3.398573
BCE tensor(538.1647, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.6485, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [120300/128000 (94%)]	Loss: 3.605875
BCE tensor(544.6807, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(273.4765, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [120450/128000 (94%)]	Loss: 3.649436
BCE tensor(550.9719, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(274.7827, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [120600/128000 (94%)]	Loss: 3.691465
BCE tensor(507.8448, device='cuda:0', grad_fn=<DivBackward0>)
KLD te

Train Epoch: 1 [126900/128000 (99%)]	Loss: 3.504475
BCE tensor(539.2080, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(269.3806, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [127050/128000 (99%)]	Loss: 3.612679
BCE tensor(499.0862, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.7159, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [127200/128000 (99%)]	Loss: 3.345355
BCE tensor(478.5642, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.9289, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [127350/128000 (99%)]	Loss: 3.208557
BCE tensor(521.9296, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(275.0562, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [127500/128000 (100%)]	Loss: 3.497868
BCE tensor(487.6801, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(265.5221, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 1 [127650/128000 (100%)]	Loss: 3.268902
BCE tensor(486.7568, device='cuda:0', grad_fn=<DivBackward0>)
KLD 

BCE tensor(483.8606, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.1814, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [6000/128000 (5%)]	Loss: 3.243816
BCE tensor(533.3906, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(270.7796, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [6150/128000 (5%)]	Loss: 3.573989
BCE tensor(508.3089, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(272.3137, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [6300/128000 (5%)]	Loss: 3.406880
BCE tensor(503.9040, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(270.3496, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [6450/128000 (5%)]	Loss: 3.377383
BCE tensor(519.5315, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(270.7523, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [6600/128000 (5%)]	Loss: 3.481593
BCE tensor(491.6806, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(269.4626, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch

Train Epoch: 2 [13050/128000 (10%)]	Loss: 3.317902
BCE tensor(547.6427, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(269.0491, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [13200/128000 (10%)]	Loss: 3.668888
BCE tensor(509.9389, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(268.9966, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [13350/128000 (10%)]	Loss: 3.417526
BCE tensor(514.6300, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(271.3026, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [13500/128000 (11%)]	Loss: 3.448953
BCE tensor(514.7987, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(273.7507, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [13650/128000 (11%)]	Loss: 3.450241
BCE tensor(517.9833, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(270.3056, device='cuda:0', grad_fn=<DivBackward0>)
Train Epoch: 2 [13800/128000 (11%)]	Loss: 3.471242
BCE tensor(533.8817, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(2

Process Process-2:
Traceback (most recent call last):
  File "/share/apps/python/2.7.12/intel/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/python/2.7.12/intel/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 187, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 164, in default_collate
    return torch.stack(batch, 0, out=out)
KeyboardInterrupt
